# Unit Commitment

This tutorial runs through examples of unit commitment for generators at a single bus. Examples of minimum part-load, minimum up time, minimum down time, start up costs, shut down costs and ramp rate restrictions are shown, as well as how to set up a rolling horizon optimization.

To enable unit commitment on a component (`Link` or `Generator`), set its attribute `committable=True`.

Remember:
1. Minimum part load $\rightarrow$ ON means must generate
2. Minimum up time $\rightarrow$ ON means must stay ON
3. Minimum down time $\rightarrow$ OFF means must stay OFF
4. Binary status $\rightarrow$ couples decisions over time

In [1]:
import pandas as pd

import pypsa

## Minimum Part Load

In the final snapshot, the load goes below the part-load limit of the coal generator (30%), forcing gas to commit.

**What PyPSA is enforcing:**

Because the generators are committable, PyPSA introduces:
1. Binary on/off status
2. Minimum generation when on
3. Startup/shutdown logic

So when a generator is on, it must respect: `p >= p_min_pu` $\times$ `p_nom`

**Snapshots 0-2: Coals stays on**

Since load is always above 3,000 MW, coal can stay committed, adjust output to exactly match load, and avoid expensive gas entirely. 

**Snapshot 3:**

If coal stayed on with its minimum output of 3,000 MW, that would over-generate by 2,200 MW. It has no place to dump power so this source is infeasible. Keeping cola on violates the minimum part load constraint. Hence, you shut down coal and turn on gas as it can produce exactly the required load. Although it has hgiher cost, it is a more feasible option. 

### Mathematical formulation (standard MILP):

**Decision variables:**

1. Generator commitment (on/off): $u_{g,t} \in {0,1}$
    * This is created automatically when ```committable=True```

2. Generator output: $p_{g,t} \geq 0$
    * Electrical output in MW, continuous variable

3. Startup/shutdown indicators:
$$y_{g,t} \in {0,1} startup at t$$
$$z_{g,t} \in {0,1} shutdown at t$$
* These track transitions, needed for minimum up/down time logic

**Minimum part load (```p_min_pu```)**
* Let $P^{max}_g$ = ```p_nom```
* Let $\underbar{p}_g$ = ```p_min_pu```
Constraint: $$\underbar{p}_g P^{max}_g u_{g,t} \leq p_{g,t} \leq P^{max}_g u_{g,t}$$
Interpretation: 
* If the generator is OFF ($u=0$): $p_{g,t} =0$
* If the generator is ON ($u=1$): $\underbar{p}_g P^{max}_ \leq p_{g,t} \leq P^{max}_g$
* This is any coal is infeasible at 800 MW load adn gas has to take over instead.

**Objective:**
$$\min \sum_t \left(c_g p_{g,t} + c_g^{standby} u_{g,t} + c_g^{start} y_{g,t}\right)$$
* $c_g =$ ```marginal_cost```
* $c_g^{standby}=$ ```stand_by_cost```
* $c_g^{start}=$ optional startup cost

In [2]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    p_nom=10_000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    p_nom=1_000,
)

nu.add("Load", "load", bus="bus", p_set=[4_000, 6_000, 5_000, 800])

In [3]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 3.13s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 36 duals
Objective: 3.56e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


('ok', 'optimal')

In [4]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0


In [5]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,4000.0,0.0
1,6000.0,0.0
2,5000.0,0.0
3,0.0,800.0


## Minimum Up Time

Gas has a minimum up time, forcing it to be online longer than otherwise necessary, which incurs a standby cost for status up without generation.

If gas starts at snapshot 1, it would have to remain on in snapshots 1, 2, 3. That would force gas to be on in snapshot 3, where coal alone can meet the load more cheaply. So PyPSA starts gas one snapshot earlier, at snapshot 0, to align the 3-period up-time window optimally.

Gas generates only 100 MW in snapshots 0 & 2 becase when it is ON, it must respect $p ≥ p_min_pu × p_nom = 100$ MW. So PyPSA keeps gas at minimum output in these snapshots, which minimizes expensive gas usage while respecting commitment constraints.

```stand_by_cost=50``` is incurred whenever gas is ON, regardless of output. It is for each snapshot separately. 

### Additional components to the mathematical formulation:

**Commitment transition equations:**

Startups and shutdowns
$$u_{g,t} -u_{g,t-1} = y_{g,y} - z_{g,t} $$

This constraint ensures:
* you can't start and stop at the same time
* Commitment status evolves correctly

**Minimum up time constraint (```min_up_time```):**

Let $UT_g=$ minimum up time (in snapshots)
$$\sum^t_{\tau=t-UT_g+1} y_{g,\tau} \leq u_{g,t} \quad \forall t$$
Interpretation:
* If a startup occured in the last $UT_g$ periods, $u_{g,t}$ must equal 1
* This is why gas stayed online even when it produced only 100 MW

**Initial conditions:**

If generator was ON before the horizon, $u_{g,t}=1$. And the remaining up time is:
$$UT_g^{remaining} = \max(0,UT_g-up\_time\_before)$$

In [2]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    p_nom=10000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    stand_by_cost=50,
    marginal_cost=70,
    p_min_pu=0.1,
    up_time_before=0,
    min_up_time=3,
    p_nom=1_000,
)

nu.add("Load", "load", bus="bus", p_set=[4_000, 800, 5_000, 3_000])

In [3]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')


INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 3.13s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 39 duals
Objective: 3.06e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-up-time, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


('ok', 'optimal')

In [8]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,1.0,1.0
1,0.0,1.0
2,1.0,1.0
3,1.0,0.0


In [9]:
nu.objective

306150.0

In [10]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,3900.0,100.0
1,0.0,800.0
2,4900.0,100.0
3,3000.0,-0.0


## Minimum Down Time

Coal has a minimum down time, forcing it to go off longer than otherwise cost-optimal.

**Constraints added:**
```
min_down_time = 2
down_time_before = 1
```

Meaning for coal:
* If coal shuts down, it must stay off for at least 2 snapshots
* At snapshot 0, coal is treated as having already been down for 1 snapshot
* So if it is off at snapshot 0, it must also be off at snapshot 1

Even though coal could meet the load in snapshot 0, it cannot be turned on, because doing so would violate the down-time constraint around snapshot 1.

NOTE: Ideally should specify ```up_time_before``` if specifying ```down_time_before=1```. 

### Additional components to the mathematical formulation:

**Minimum down time constraint (```min_down_time```)**

Let $DT_g =$ minimum down time
$$\sum^t_{\tau=t-DT_g+1} z_{g,\tau} \leq 1-u_{g,t} \quad \forall t$$
Interpretation:
* If a shutdown happened within the last $DT_g$ periods, $u_{g,t}$ must equal 0.
* This is what forced coal to stay off for two full snapshots, even though it was cheap.

**Initial conditions:**

If generator was OFF before the horizon, $u_{g,0}=0$. And the remaining down time is:
$$DT_g^{remaining} = \max(0,DT_g-down\_time\_before)$$

In [4]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    min_down_time=2,
    down_time_before=1,
    p_nom=10_000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    p_nom=4_000,
)

nu.add("Load", "load", bus="bus", p_set=[3_000, 800, 3_000, 8_000])

In [5]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.06s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 40 duals
Objective: 4.86e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-down-time, Generator-com-status-min_up_time_must_stay_up, Generator-com-status-min_down_time_must_stay_up were not assigned to the network.


('ok', 'optimal')

In [6]:
nu.objective

486000.0

In [7]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


In [8]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,0.0,3000.0
1,0.0,800.0
2,3000.0,0.0
3,8000.0,0.0


## Start Up and Shut Down Costs

Now there are costs associated with shut down and start up events, which could incentivise longer up times of generators with high start-up and shut-down costs.

Introduce non-convex transition costs:
* Coal has a large start-up cost $\rightarrow$ it is expensive to turn ON. The optimizer prefers fewer startups and longer continuous periods. EVen if cola is cheaper per MWh, it is expensive per event.
* Gas has a shutdown cost $\rightarrow$ it is expensive to turn OFF. Gas prefers to stay ON during low-load periods, possibly run at minimum output.

The optimizer decides *if it is cheaper to keep a generator running through low laod periods, or shut it down and pay the transition cost?*

### Additional components to the mathematical formulation:

**New objective function:**

Let $C_g^{SU} =$ ```start_up_cost``` and $C_g^{SD} =$ ```shut_down_cost```. The new objective function is
$$\min \sum_t \left(c_g p_{g,t} + C_g^{SU} y_{g,t} + C_g^{SD} z_{g,t} \right)$$

Interpretation: 
1. If generator starts, the system operator pays $C_g^{SU}$
2. If generator shuts down, the system operator pays $C_g^{SD}$
3. If genreator stays ON or OFF, there is not transition cost.

In [2]:
nu = pypsa.Network(snapshots=range(4))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    min_down_time=2,
    start_up_cost=5_000,
    p_nom=10_000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    shut_down_cost=25,
    p_nom=4_000,
)

nu.add("Load", "load", bus="bus", p_set=[3_000, 800, 3_000, 8_000])

In [3]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 2.61s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 32 primals, 39 duals
Objective: 4.91e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-down-time, Generator-com-status-min_up_time_must_stay_up were not assigned to the network.


('ok', 'optimal')

In [4]:
nu.objective

491025.0

In [5]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


In [6]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,0.0,3000.0
1,0.0,800.0
2,3000.0,0.0
3,8000.0,0.0


## Ramp rate limits

Ramp rate limits can be set for ramping up and down and are given as percentage of the nominal power that can be ramped up or down per snapshot. Note that the ramp limits apply per snapshot and are not weighted by the time step duration (```nu.snapshot_weightings```).

**What ramp rate limit means:**

A ramp rate limit restricts how fast a generator's output can change between consecutive snapshots. In PyPSA:
* Ramp limits are fractions of nominal power per snapshot
* They are not time-scaled by snapshot length

So if: ```p_nom = 10,000 MW``` and ```ramp_limit_up = 0.1```, $\Delta p \leq 1000$ MW per snapshot

### Additional components to the mathematical formulation:

#### Ramp limits - non-commitable generators (first two examples below):

Let,
* $RU_g=$ ```ramp_limit_up```
* $RD_G=$ ```ramp_limit_down```
* $P_g^{max}=$ ```p_nom``` (or ```p_nom_opt``` if extendable)

Ramp-up constraint: You cannot increase output faster than the ramp-up limit.
$$p_{g,t} - p_{g,t-1} \leq RU_g \cdot P_g^{max}$$

Ramp-down constraint: You cannot decrease output faster than the ramp-down limit.
$$p_{g,t-1} - p_{g,t} \leq RR_g \cdot P_g^{max}$$

#### Ramp limits with capacity expansion:

When ```p_nom_extensable=True```, $P_g^{max}$ becomes a decision variable. So ramp constraints become bilinear-looking, by PyPSA keeps them linear by coupling both variables:
$$p_{g,t} - p_{g,t-1} \leq RU_g \cdot P_g^{max}$$
Not the optimizer can:
* Increse capacity
* Relax ramp bottlenecks
* Trade capital cost vs operational flexibility

That's why coal exapnds to 5000 MW:
* Enough to meet load
* Enough ramp margin to avoid gas use

#### Ramp limits with committable generators:

If a unit can turn ON/OFF, ramping must respect status changes.

**Regular ramping (when ON in both periods)**
$$p_{g,t} - p_{g,t-1} \leq RU_g \cdot P_g^{max} + M \cdot (1-u_{g,t-1})$$
$$p_{g,t-1} - p_{g,t} \leq RD_g \cdot P_g^{max} + M \cdot (1-u_{g,t})$$
* Here, $M$ is a "big-M" term that disables the constraint when unit is OFF
* Interpretation: Ramp limits apply only when the unit if ON

**Start-up ramp limit (```ramp_limit_start_up```):**
* $SU_g=$ ```ramp_limit_start_up```
$$p_{g,t} \leq SU_g \cdot P_g^{max} \cdot y_{g,t}$$
* When starting up, output cannot exceed start-up ramp limit.

**Shut-down ramp limit (```ramp_limit_shut_down```):**
* $SD_g=$ ```ramp_limit_shut_down```

#### Interaction with minimum part load (CRITICAL)
Minimum part load: $$p_{g,t} \geq p_g^{min} P_g^{max}u_{g,t}$$
Start-up ramp: $$p_{g,t} \leq SU_gP_g^{max}$$
Feasibility condition: $$SU_g \geq p_g^{min}$$
Otherwise:
* You cannot ramp up fast enough to reach minimum stable output
* Generator cannot start $\rightarrow$ infeasible

### Why "bad interactions" happen (warning below)
Example problem
```
p_min_pu = 0.05
ramp_limit_start_up = 0.1
ramp_limit_up = 0.2
```
This looks safe - but:
* Load is low immediately after startup
* Ramp-down may be tigher than min load
* Shutdown ramp may conflict with ramp-down

**General feasibility rules:**
$$SU_g \geq _g^{min}$$
$$SD_g \geq p_g^{min}$$
$$RU_g \geq p_G^{min}$$
Otherwise:
* Unit can be forced ON but unable to generate
* Or forced OFF but unable to ramp down

In [7]:
nu = pypsa.Network(snapshots=range(6))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    marginal_cost=20,
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    p_nom=10_000,
)

nu.add("Generator", "gas", bus="bus", marginal_cost=70, p_nom=4_000)

nu.add("Load", "load", bus="bus", p_set=[4_000, 7_000, 7_000, 7_000, 7_000, 3_000])

In [8]:
nu.optimize(log_to_console=False)   

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.04s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 12 primals, 40 duals
Objective: 9.50e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-fix-p-ramp_limit_up, Generator-fix-p-ramp_limit_down were not assigned to the network.


('ok', 'optimal')

In [10]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,4000.0,-0.0
1,5000.0,2000.0
2,6000.0,1000.0
3,7000.0,-0.0
4,5000.0,2000.0
5,3000.0,-0.0


With capacity expansion (as long as unit is not committable):

In [11]:
nu = pypsa.Network(snapshots=range(6))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    marginal_cost=20,
    ramp_limit_up=0.1,
    ramp_limit_down=0.2,
    p_nom_extendable=True,
    capital_cost=1e2,
)

nu.add("Generator", "gas", bus="bus", marginal_cost=70, p_nom=4000)

nu.add("Load", "load", bus="bus", p_set=[4000, 7000, 7000, 7000, 7000, 3000])

In [12]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.05s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 13 primals, 41 duals
Objective: 1.68e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Generator-ext-p-ramp_limit_up, Generator-ext-p-ramp_limit_down were not assigned to the network.


('ok', 'optimal')

In [13]:
nu.generators.p_nom_opt

name
coal    5000.0
gas     4000.0
Name: p_nom_opt, dtype: float64

In [14]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,4000.0,-0.0
1,4500.0,2500.0
2,5000.0,2000.0
3,5000.0,2000.0
4,4000.0,3000.0
5,3000.0,-0.0


Watch out for bad interactions, for example, when the ramp limit at start up or shut down is bigger than the regular ramp limit or minimum part load, which can lead to infeasibilities.

In [15]:
nu = pypsa.Network(snapshots=range(7))

nu.add("Bus", "bus")

# Can get bad interactions if SU > RU and p_min_pu; similarly if SD > RD
nu.add(
    "Generator",
    "coal",
    bus="bus",
    marginal_cost=20,
    committable=True,
    p_min_pu=0.05,
    initial_status=0,
    ramp_limit_start_up=0.1,
    ramp_limit_up=0.2,
    ramp_limit_down=0.25,
    ramp_limit_shut_down=0.15,
    p_nom=10_000,
)

nu.add("Generator", "gas", bus="bus", marginal_cost=70, p_nom=10_000)

nu.add("Load", "load", bus="bus", p_set=[0, 200, 7_000, 7_000, 7_000, 2_000, 0])

In [16]:
nu.optimize(log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.07s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 35 primals, 61 duals
Objective: 1.15e+06
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-status-min_up_time_must_stay_up, Generator-com-p-ramp_limit_up, Generator-com-p-ramp_limit_down were not assigned to the network.


('ok', 'optimal')

In [17]:
nu.generators_t.p

name,coal,gas
snapshot,,
0,0.0,0.0
1,0.0,200.0
2,1000.0,6000.0
3,3000.0,4000.0
4,4000.0,3000.0
5,1500.0,500.0
6,0.0,0.0


In [18]:
nu.generators_t.status

name,coal,gas
snapshot,,
0,0.0,0.0
1,0.0,0.0
2,1.0,0.0
3,1.0,0.0
4,1.0,0.0
5,1.0,0.0
6,0.0,0.0


## Rolling Horizon

The unit commitment optimisation can be combined with a rolling horizon optimisation, i.e. solving the snapshots sequentially in batches. This can be done manually (as shown here) or automatically, using nu.optimize.```optimize_with_rolling_horizon()```.

***Note:*** Full unit commitment (UC) with ramping, startup/shutdown costs, and rolling-horizon decomposition

#### Base Unit Commitment Mathematical Formulation:

**Sets:**
* $g\in \mathcal{G}:$ generators (coal, gas)
* $t\in \mathcal{T}:$ time snapshots

**Decision Variables:**

For each generator $g$ and time $t$:
* $p_{g,t} \geq 0 \rightarrow$ continuous, power output
* $u_{g,t} \in {0,1} \rightarrow$ binary, on/off status
* $v_{g,t} \in {0,1} \rightarrow$ binary, start-up indicator
* $w_{g,t} \in {0,1} \rightarrow$ shut-down indicator

**Parameters (from the code):**
* $\bar{P}_g :$ nominal capacity (```p_nom```)
* $\underbar{p}_g :$ minimum part load (```p_min_pu```)
* $c_g :$ marginal cost 
* $C_g^{SU} :$ start-up cost
* $C_g^{SD} :$ shut-down cost
* $T_g^{up} :$ minimum up time
* $T_g^{down} :$ minimum down time
* $R_g^{up}, R_g^{down} :$ ramp limits
* $R_g^{SU}, R_g^{down} :$ start-up / shut-down ramps
* $d_t :$ load demand

#### Objective function:
Minimize total system cost:
$$\min \sum_{t\in\mathcal{T}} \sum_{g\in\mathcal{G}} \left(c_g p_{g,t} + C_g^{SU}v_{g,t} + C_g^{SD}w_{g,t} \right)$$
* Energy cost: $c_g p_{g,t}$
* Penalty for turning units on/off
* Encourages fewwer transitions for expensive units (coal)

#### Core Constraints:

(A) Power Balance (System Constraint):
$$\sum_g p_{g,t} = d_t \forall t$$

(B) Capacity & Minimum Part load: These implement ```p_min_pu```
$$\underbar{p}_g \bar{P}_g u_{g,t} \leq p_{g,t} \leq \bar{P}_g u_{g,t} \forall g,t$$
Meaning
* If OFF $(u=0) \rightarrow p=0$
* If ON $(u=1) \rightarrow$ must produce at least minimum stable output

(C) Unit State Transitions:
$$u_{g,t} - u_{g,t-1} = v_{g,t} w_{g,t} \forall g,t$$
Meaning:
* Start-up when switching $0\rightarrow 1$
* Shut-down when switching $1\rightarrow 0$

This is the backbone of all time-coupled logic

#### Minimum Up-time Constraint:
Implements ```min_up_time=3```
$$\sum_{\tau=t}^{t+T_g^{up}-1} (1-u_{g,\tau}) \geq T_g^{up} w_{g,t} \quad \forall g,t$$
Interpretation:
* If you start at time $t$, you must stay ON for at least 3 periods
* prevents short-cycling

#### Minimum Down-time Constraint:
Implements ```min_down_time=2```
$$\sum_{\tau=t}^{t+T_g^{down}-1} (1-u_{g,\tau}) \geq T_g^{down} w_{g,t} \quad \forall g,t$$
Interpretation:
* After shutdown, unit must remain OFF for 2 periods
* Important for thermal units

#### Ramping Constraint:
**Normal ramping:**
$$p_{g,t} - p_{g,t-1} \leq R_g^{up} \bar{P}_g \quad \forall g,t $$
$$p_{g,t-1} - p_{g,t} \leq R_g^{down} \bar{P}_g \quad \forall g,t $$

**Start-up ramp:**
$$p_{g,t} \leq R_g^{SU} \bar{P}_g v_{g,t} + \bar{P}_g u_{g,t-1} $$

**Shut-down ramp:**
$$p_{g,t-1} \leq R_g^{SD} \bar{P}_g w_{g,t} + \bar{P}_g u_{g,t} $$
This ensures realistic transitions during on/off events

#### Initial Conditions (Rolling Horizon Critical):
From parameters like ```up_time_before``` and ```down_time_before```

These define: $u_{g,0},$ remaining up/down time. They link optimization windows together.

#### Rolling Horizon Formulation
Instead of solving over all $\mathcal{T}:$

**Global UC (ideal but expensive)** $$\min_{u,p,v,w} \sum_{t=1}^T costs$$

**Rolling Horizon UC (what is done here)** 

You solve subproblems: $$\mathcal{T}_k = {kL, ..., kL+H} $$
Where:
* L: step length (```len(p_set)```)
* H: overlap (```overlapp=2```)

Each subproblem: $$\min \sum_{t\in \mathcal{T}_k} costs$$
Subject to:
* UC constraints
* Initial state fixed from pervious solution

Only decision for $[kL, (k+1)L -1]$ are committed

The overlap ensures:
* Minimum up/down time constraints are not violated
* Ramp constraints are respected across windows

Mathematically:
* Variables in overlap are look-ahead only
* First $L$ steps are finalized

#### Algorithmic View (What PyPSA is doing)

```
snapshots = nu.snapshots[i * L : (i + 1) + L + overlap]
nu.optimize(snapshots = snapshots)
```
Equivalent to:
1. Fix previous generator states
2. Solve a MILP UC problem
3. Store terminal states
4. Slide horizon forward
5. Repeat

This is Model Predictive Control (MPC) applied to unit commitment.



In [2]:
sets_of_snapshots = 6
p_set = [4_000, 5_000, 700, 800, 4_000]

nu = pypsa.Network(snapshots=range(len(p_set) * sets_of_snapshots))

nu.add("Bus", "bus")

nu.add(
    "Generator",
    "coal",
    bus="bus",
    committable=True,
    p_min_pu=0.3,
    marginal_cost=20,
    min_down_time=2,
    min_up_time=3,
    up_time_before=1,
    ramp_limit_up=1,
    ramp_limit_down=1,
    ramp_limit_start_up=1,
    ramp_limit_shut_down=1,
    shut_down_cost=150,
    start_up_cost=200,
    p_nom=10_000,
)

nu.add(
    "Generator",
    "gas",
    bus="bus",
    committable=True,
    marginal_cost=70,
    p_min_pu=0.1,
    up_time_before=2,
    min_up_time=3,
    shut_down_cost=20,
    start_up_cost=50,
    p_nom=1_000,
)

nu.add("Load", "load", bus="bus", p_set=p_set * sets_of_snapshots)

In [3]:
overlap = 2
for i in range(sets_of_snapshots):
    snapshots = nu.snapshots[i * len(p_set) : (i + 1) * len(p_set) + overlap]
    nu.optimize(snapshots=snapshots, log_to_console=False)

Index(['bus'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 2.52s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 56 primals, 96 duals
Objective: 5.55e+05
Solver model: available
Solver message: Optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-com-p-lower, Generator-com-p-upper, Generator-com-transition-start-up, Generator-com-transition-shut-down, Generator-com-up-time, Generator-com-down-time, Generator-com-status-min_up_time_must_stay_up, Generator-com-p-ramp_limit_up, Generator-com-p-ramp_limit_down were not assigned to the network.
Index(['bus'], dtype='object', name='name')
Index(['0'], dtype='object', name='name')
INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.model:Solver options:
 - log_to_console: False
INFO:linopy.io: Writing time: 0.06s
INFO:l

In [4]:
pd.concat(
    {"Active": nu.generators_t.status.astype(bool), "Output": nu.generators_t.p}, axis=1
)

Active         Output       
name       coal    gas    coal    gas
snapshot                             
0          True   True  3900.0  100.0
1          True   True  4900.0  100.0
2         False   True     0.0  700.0
3         False   True     0.0  800.0
4          True  False  4000.0    0.0
5          True  False  4000.0    0.0
6          True  False  5000.0    0.0
7         False   True     0.0  700.0
8         False   True     0.0  800.0
9          True   True  3900.0  100.0
10         True  False  4000.0    0.0
11         True  False  5000.0    0.0
12        False   True     0.0  700.0
13        False   True     0.0  800.0
14         True   True  3900.0  100.0
15         True  False  4000.0    0.0
16         True  False  5000.0    0.0
17        False   True     0.0  700.0
18        False   True     0.0  800.0
19         True   True  3900.0  100.0
20         True  False  4000.0    0.0
21         True  False  5000.0    0.0
22        False   True     0.0  700.0
23        False   True     0.0  800.0
24         True   True  3900.0  100.0
25         True  False  4000.0    0.0
26         True  False  5000.0    0.0
27        False   True     0.0  700.0
28        False   True     0.0  800.0
29         True   True  3900.0  100.0